# Lab 1: 'シンギュラリティ機械学習 - 分類' by Multiverse Computing

「シンギュラリティ機械学習 - 分類」function の力を探るこのハンズオンラボへようこそ。このセッションでは、古典的手法と量子強化技術の両方を活用した実社会の機械学習課題であるグリッド安定性分類を解説します。scikit-learnの前処理とIBMのサーバーレスインフラ上でホストされている量子アンサンブル分類器をシームレスに統合したハイブリッドワークフローの経験を積めます。シミュレーションの実行でも実際の量子ハードウェアでも、このラボではスケーラブルな量子機械学習が現実世界の応用をどのように加速できるかを実践的に見ることができます。単にモデルを構築するだけでなく、このノートの目的はQuantumEnhancedEnsembleClassifierの機能を理解し、その機能や利点について直感を育む手助けをすることです。まずAdaBoostを使って古典的なベンチマークを設定し、その後、量子分類器をシミュレートされた量子バックエンドで実行すること、多様な学習者による異種アンサンブルのシミュレーション、そして実際の量子ハードウェア上で分類器を実行するという3つの主要な機能を示す演習を進めます。各ステップは、ツールに対する信頼を深め、量子最適化がどこでどのように意味のある利益をもたらすかの理解を深めることを目的としています。さあ、始めましょう!

# Table of Contents

* [Setup](#setup)
* [Imports](#imports)
* [Initialize the 'Singularity Machine Learning - Classification' Function](#initialize-the-singularity-machine-learning---classification-function)
    * [Set Static Variables and IBM Token](#set-static-variables-and-ibm-token)
    * [Load the 'Singularity Machine Learning - Classification' Function and Client](#load-the-singularity-machine-learning---classification-function-and-client)
* [Define Helper Functions](#define-helper-functions)
* [Load the Dataset](#load-the-dataset)
* [AdaBoostClassifier](#adaboostclassifier)
* [QuantumEnhancedEnsembleClassifier](#quantumenhancedensembleclassifier)
    * [Introduction](#introduction)
  * [Understanding the Optimization Parameters](#understanding-the-optimization-parameters)
* [Part 1: Simulated Quantum Classifier](#part-1-simulated-quantum-classifier)
    * [Execute the Job](#execute-the-job)
    * [Monitor Job Execution](#monitor-job-execution)
    * [Retrieve and Analyze the Results](#retrieve-and-analyze-the-results)
* [Part 2: Simulated Heterogenous Quantum Classifier](#part-2-simulated-heterogenous-quantum-classifier)
    * [Execute the Job](#execute-the-job)
    * [Monitor Job Execution](#monitor-job-execution)
    * [Retrieve and Analyze the Results](#retrieve-and-analyze-the-results)
* [Part 3: Quantum Classifier on Real Quantum Hardware](#part-3-quantum-classifier-on-real-quantum-hardware)
    * [Execute the Job](#execute-the-job)
    * [Monitor Job Execution](#monitor-job-execution)
    * [Retrieve and Analyze the Results](#retrieve-and-analyze-the-results)
    * [Visualization: Classical vs Quantum F1 Score](#visualization-classical-vs-quantum-f1-score)
* [Clean up Shared Directory](#clean-up-shared-directory)
* [Scaling Benefits and Enhancements](#scaling-benefits-and-enhancements)
* [References](#references)
* [Additional information](#additional-information)
* [Qiskit packages versions](#qiskit-packages-versions)


# Setup

In [ ]:
# Install dependencies
%pip install "qiskit[visualization]~=2.1.0" "qiskit-serverless~=0.24.0" "qiskit-ibm-catalog~=0.8.0" "scikit-learn==1.5.2" "pandas>=2.0.0,<3.0.0" "imbalanced-learn~=0.12.3" 

In [ ]:
# Imports

import sys
import os
import tarfile
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from qiskit_serverless import IBMServerlessClient, QiskitFunction
from typing import Tuple
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier
from grader import grade_exercise_1, grade_exercise_2, grade_exercise_3

<div class="alert alert-block alert-warning">

**Qiskit Functionsへの独占アクセス**

Qiskit GLobal Summer School (QGSS)の一環として、プレミアムまたはフレックスプランの参加者はQiskit Functions に限定期間トライアルでアクセスできます。アクセスは独占的で、組織の管理者の承認が必要です。アクセスを申請するには[このフォーム](https://airtable.com/appj8IrSNZGz4l4BB/pag8WgWdUr5uSJGZA/form)にご記入ください。

もし、下のセルで「`QiskitServerlessException: Credentials couldn't be verified`というエラーに遭遇した場合は、Qiskit Functions へのアクセスがまだ有効でないことを意味します。申請が処理された後、またご確認ください。

**注:このラボの運用には、貴組織のアカウントからQPU時間が消費されます。推定QPU使用量は、QPU上で実行される各セルの前に提供されます。QGSS機能ラボの割り当てQPU時間について不明な場合は、使用状況を監視し、組織管理者にご相談ください。**


</div>

# 「シンギュラリティ機械学習 - 分類」Functionの初期化

まずIBM Qiskit Serverless client を初期化し、Multiverse ComputingワークスペースからFunctionを取得します。Qiskit Serverlessは、基盤となるインフラを気にせずに柔軟かつスケーラブルなQiskitプログラムを実行する方法を提供します。複雑な量子古典的ワークフローをクラウドにオフロードし、CPU、GPU、QPU間でリソースを自動的に管理できます。これにより、IBM Quantum Platformでのリモート管理実行の恩恵を受けつつ、アルゴリズムのロジックに集中しやすくなります。

「シンギュラリティ機械学習 - 分類」Functionを読み込むことで、ハイブリッドワークフローに使える強力な量子強化分類器にアクセスできます。このFunctionは古典的成分と量子成分の両方を包含し、量子処理によって強化されたアンサンブル手法で訓練、最適化、予測を可能にします。詳細は当社のドキュメントをご参照ください:<https://docs.quantum.ibm.com/guides/multiverse-computing-singularity>。

まずはサーバーレス環境に接続し、Functionを読み込みます。

In [ ]:
print("Using IBM Qiskit Serverless...\n")

# Load the IBM Serverless Client
your_api_key = "deleteThisAndPasteYourAPIKeyHere"
your_crn = "deleteThisAndPasteYourCRNHere"

client = IBMServerlessClient(
    channel="ibm_quantum_platform",
    token=your_api_key,
    instance=your_crn,
)
# You should see a list of Qiskit Functions available to you
# If you encounter the error `QiskitServerlessException: Credentials couldn't be verified`,
# it means your access is not yet active
client.list()

In [ ]:
# Load the Multiverse Singularity function

singularity = client.get("multiverse/singularity")

In [ ]:
# Define static variables

RANDOM_STATE: int = 123
TRAIN_PATH = "data/grid_stability/train.csv"
TEST_PATH = "data/grid_stability/test.csv"

# ヘルパー関数の定義

ワークフローを効率化するために、このラボ全体でデータ処理と評価に使うヘルパー関数のセットを定義しています。これらの機能により、ローカルデータ処理とQiskitサーバーレスでのリモート実行の円滑なインターフェースが可能になります。


In [ ]:
def load_data(data_path: str) -> Tuple[np.ndarray, np.ndarray]:
    """Load data from the given path to X and y arrays."""
    df: pd.DataFrame = pd.read_csv(data_path)
    return df.iloc[:, :-1].values, df.iloc[:, -1].values


def make_tarfile(file_path, tar_file_name):
    """Create a tar file from the given file."""
    with tarfile.open(tar_file_name, "w") as tar:
        tar.add(file_path, arcname=os.path.basename(file_path))


def upload_data(name: str, data: np.ndarray, client: IBMServerlessClient, function: QiskitFunction):
    """Save the data to a file, create a tar file, upload it, and remove the files."""
    np.save(name, data)
    make_tarfile(name, f"{name}.tar")
    client.file_upload(f"{name}.tar", function)

    
def evaluate_predictions(predictions, y_true, start_time=None, end_time=None):
    accuracy = accuracy_score(y_true, predictions)
    precision = precision_score(y_true, predictions)
    recall = recall_score(y_true, predictions)
    f1 = f1_score(y_true, predictions)
    if start_time is not None and end_time is not None:
        print("Time taken (s):", end_time - start_time)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1:", f1)
    return accuracy, precision, recall, f1

# データセットの読み込み

次に、量子強化分類器の訓練と評価に使用したデータセットを読み込み、前処理します。このデータセットは、センサー入力に基づいてシステムの安定性を予測するグリッド安定性分類タスクのためのラベル付きサンプルで構成されています。

まずCSVファイルからトレーニングデータとテストデータを読み取り、さらにトレーニングセットを分割して検証セットを作成します。実際のデータセットでよくあるクラスの不均衡に対処するため、少数派クラスを補強するためにランダムなオーバーサンプリングを適用し、分類器が支配的なラベルに偏らないようにします。

準備が完了すると、データセットはNumPy配列としてシリアライズされ、Qiskitサーバーレス環境にアップロードされます。これにより、リモートの「シンギュラリティ機械学習 - 分類」function が分散実行中にデータにアクセスし、操作できるようになります。

In [ ]:
# Load and upload the data
X_train, y_train = load_data(TRAIN_PATH)
X_test, y_test = load_data(TEST_PATH)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=RANDOM_STATE
)
# Balance the dataset through over-sampling of the positive class
ros = RandomOverSampler(random_state=RANDOM_STATE)
X_train, y_train = ros.fit_resample(X_train, y_train)

In [ ]:
print("-- Uploading data --")
upload_data("grid_stability_X_train.npy", X_train, client, singularity)
upload_data("grid_stability_y_train.npy", y_train, client, singularity)
upload_data("grid_stability_X_test.npy", X_test, client, singularity)
print("Data uploaded!\n")

# AdaBoostClassifier

データが整ったところで、まずは古典的機械学習で確立されたアンサンブル手法であるAdaBoostClassifierを使いましょう。AdaBoostは複数の弱学習者を単一の強力な分類器に組み合わせることで動作し、分類が難しいサンプルにより重点を置くことで性能を反復的に向上させます。

このラボでは、バランストレーニングセットを用いてAdaBoostモデルを訓練し、テストセット上で評価します。これは、後で使う量子強化分類器との比較に強力な基準を提供します。推定量の数は75に設定されており、アンサンブルが過学習せずによく一般化できるようにしています。

トレーニング後、テストセット上で予測を生成し、精度、精度、リコール率、F1スコアといった主要な性能指標を計算して、このグリッド安定性分類タスクにおけるモデルの性能を評価します。

In [ ]:
classifier = AdaBoostClassifier(n_estimators=75, random_state=RANDOM_STATE)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)

In [ ]:
evaluate_predictions(predictions, y_test)

# QuantumEnhancedEnsembleClassifier

### はじめに

 `QuantumEnhancedEnsembleClassifier` は、Multiverse Computingの 'Singularity Machine Learning - Classification' Function によるハイブリッド機械学習モデルで、ブーストやバギングといった古典的なアンサンブル手法とQAOAのような量子最適化アルゴリズムを組み合わせています。学習者数が増えるにつれて訓練コストが増加する古典的アンサンブルとは異なり、この量子分類器は良好なスケーラビリティ特性を示します。学習者数(および量子ビット数)が増えるにつれて、訓練時間は比較的安定します。このため、特に大規模なアンサンブルを必要とする問題に適しており、一方、古典モデルは指数関数的にスケールする最適化コストに苦労しがちです。さらに、QSVMのような従来の量子モデルがデータセットサイズに制約されることが多いのに対し、この分類器は主に利用可能なハードウェアインフラによって制約されるデータポイントや特徴の数に依存しないよう設計されており、大規模かつ高次元のデータセットの処理に適しています。量子ハードウェアがスケールアップするにつれて、このモデルの精度と性能も向上し、将来の現実世界の量子優位性への魅力的な軌道を提供します。本ノートでは、古典シミュレーションと実際の量子実行の両方での利用を示し、その可能性と柔軟性を示しています。

![Diagram](diagram.png)

---

## 最適化パラメータの理解

`QuantumEnhancedEnsembleClassifier` を効果的に活用するには、モデルの訓練や最適化に影響を与えるいくつかの重要なパラメータの役割を理解することが重要です。この関数の使用法やパラメータの詳細については、[ドキュメント](https://docs.quantum.ibm.com/guides/multiverse-computing-singularity)を参照してください

このラボでも用いられる主なパラメータのいくつかを以下に説明します:

- **`regularization`**  
  このパラメータは最終的なアンサンブルの複雑さを制御します。高い値は大きな学習者の組み合わせや複雑な組み合わせにペナルティを与え、単純なモデルを促進し過学習リスクを減らします。 

- **`num_solutions`**  
  オプティマイザが評価する候補アンサンブル構成の数を定義します。値が高いほどより包括的な探索が可能になり、より高性能なアンサンブルを発見できますが、その代償として実行時間は長くなります。デモンストレーションとして、探索と性能のバランスを取る価値は`100,000` 程度です。

- **`simulator`**  
  これを `True` に設定すると古典的最適化が可能になります。これは高速で迅速な実験に適していますが、量子計算のスケーラビリティの利点は享受できません。これを `False` に設定するとQAOAによる量子最適化が活性化され、大規模なアンサンブルに理想的です。

- **`classical_optimizer_options`**  
  これらのオプションは、QAOAで使用される基礎となる古典的最適化装置の挙動を定義します。例えば、 `{"maxiter": 10}` は最適化回数を制限します。これは短いデモには有用ですが、実際の応用では通常 `60+` 回数で収束が改善されます。

これらのパラメータにより、速度、精度、計算リソースの使用をバランスさせる柔軟性が得られます。以下のセクションでは、古典的設定と量子設定の両方を試し、これらのトレードオフが実際にどのように機能するかを見ていきます。


# Part 1: Simulated Quantum Classifier

### Jobの実行

では、 Singularity Qiskit Functionが提供する `QuantumEnhancedEnsembleClassifier` を使って実験を始めましょう。この分類器は、古典的なアンサンブル学習と量子に触発された最適化を統合し、複数の学習者を賢く組み合わせます。

この演習では、シミュレータを使って古典的最適化を用いるモデルを設定します。これにより、実際の量子ハードウェアを呼び出さずに最適化ロジックを検証できます。デモンストレーションのために、 `num_learners`を`15`に設定し、これがアンサンブルでどれだけの弱い学習者を組み合わせるかを制御します。なお、シミュレーターモードはスケーラビリティを重視して設計されていません。 `num_learners` の値が大きいと実行時間が大幅に増加したり、パフォーマンス制限に到達したりする可能性があります。

このセットアップは、量子最適化でスケールアップする前にワークフローを迅速かつ低コストで検証し、期待されるパフォーマンスの洞察を得る方法を提供します。



<div class="alert alert-block alert-success">
  <strong>Exercise 1:</strong>  
　　<code>#TODO</code>セクションを埋めて、古典最適化のためにシミュレーターをEnableしてください。
</div>

<div class="alert alert-block alert-info">
  <strong>💡 Tip:</strong> 
  シミュレーターのEnableの仕方はここのドキュメントを参照してください：<a href="https://docs.quantum.ibm.com/guides/multiverse-computing-singularity" target="_blank">here</a> 
</div>

<div class="alert alert-block alert-warning">
  <strong>⚠️ 警告:</strong> この演習では、QPU時間の消費を避けるため、シミュレータを使用してください。
</div>

In [ ]:
optimizer_options_ex1 = {
    ### TODO: Write your code below here ###
    
    ### Don't change any code past this line ###
    "num_solutions": 100000,
}

In [ ]:
grade_exercise_1(optimizer_options_ex1)

In [ ]:
# Create, fit, and predict
print("-- Creating, fitting, and predicting --")
start = time.time()
job = singularity.run(
    action="create_fit_predict",
    name="classifier_for_grid_stability",
    quantum_classifier="QuantumEnhancedEnsembleClassifier",
    num_learners=15, 
    regularization=15,
    optimizer_options=optimizer_options_ex1,
    backend_name= None, 
    instance=your_crn,
    random_state=RANDOM_STATE,
    X_train="grid_stability_X_train.npy",
    y_train="grid_stability_y_train.npy",
    X_test="grid_stability_X_test.npy",
    fit_params={
        "validation_data": (X_val, y_val),
    },
    options={
        "save": False,
    }
)

### ジョブ実行のモニター

ジョブが送信されたので、`job` を表示して JobID を保存 できます。また、`job.status()` を使うことで 実行状況（ステータス）を確認 できます。このセルは再実行することで、最新のステータスを再取得（ポーリング） できます。もしジョブが失敗した場合は、`job.error_message()` を使って 失敗の原因に関する情報を取得 できます。

In [ ]:
job

In [ ]:
job.status()

In [ ]:
error_message = job.error_message()
print(error_message)

### 結果の取得と解析

ジョブが提出・完了したら、`job.result()`を使って結果を取得できます。その後、事前に定義したヘルパー関数を使って結果を評価し、期待に合致しているか確認できます。

In [ ]:
# Get the job status, logs, and result
result = job.result()
status = job.status()
error_message = job.error_message()
end = time.time()

print("Job status: ", status)
print("Error message: ", error_message)
print("Action status: ", result["status"])
print("Action message: ", result["message"])
print("Action result: ", result["data"])

In [ ]:
predictions = result["data"]["predictions"]
evaluate_predictions(predictions, y_test, start, end)

# Part 2: 異種量子分類機をシミュレートする 

### Jobの実行
この節では、古典的資源を用いて異種量子アンサンブルをシミュレートします。標準的なシミュレート分類器とは異なり、異種バリアントは決定木、k-近傍、ロジスティック回帰など、アンサンブル内で異なる学習器タイプを組み合わせて定義でき、それぞれが独自の帰納バイアスを寄与させます。この戦略は、特に特徴関係が多様な複雑なデータセットにおいて、モデルの多様性を高め、一般化を改善できます

<div class="alert alert-block alert-success">
  <strong>Exercise 2:</strong>  
  <code>#TODO</code>セクションを埋めて、2つのユニークな学習者を均等に分配する仕事を作成してください。
</div>

<div class="alert alert-block alert-info">
  <strong>💡 Tip:</strong> 異なる学習器の指定方法については、<a href="https://docs.quantum.ibm.com/guides/multiverse-computing-singularity" target="_blank">ドキュメント</a>を参照してください。これが結果にどのように影響したか観察してください。Exercise の graderをpassしたら、さまざまな学習器で試してみて、それがパフォーマンスにどう影響するかを学んでみてください。
</div>

In [ ]:
### TODO: Write your code below here ###
learners_types_solution =
learners_proportions_solution =
### Don't change any code past this line ###

In [ ]:
grade_exercise_2(learners_types_solution, learners_proportions_solution)

<div class="alert alert-block alert-warning">
  <strong>⚠️ 警告:</strong> この演習では、QPUの実行時間を消費しないように、今回もシミュレータを使用します。
</div>

In [ ]:
# Create, fit, and predict
print("-- Creating, fitting, and predicting --")
start = time.time()
job = singularity.run(
    action="create_fit_predict",
    name="classifier_for_grid_stability",
    quantum_classifier="QuantumEnhancedEnsembleClassifier",
    num_learners=15, 
    regularization=15,
    learners_types=learners_types_solution,
    learners_proportions=learners_proportions_solution,
    optimizer_options={
        "num_solutions": 100000,
        "simulator": True, 
    },
    backend_name= None, 
    instance=your_crn,
    random_state=RANDOM_STATE,
    X_train="grid_stability_X_train.npy",
    y_train="grid_stability_y_train.npy",
    X_test="grid_stability_X_test.npy",
    fit_params={
        "validation_data": (X_val, y_val),
    },
    options={
        "save": False,
    }
)

### ジョブ実行のモニター

ジョブが送信されたので、`job` を表示して JobID を保存 できます。また、`job.status()` を使うことで 実行状況（ステータス）を確認 できます。このセルは再実行することで、最新のステータスを再取得（ポーリング） できます。もしジョブが失敗した場合は、`job.error_message()` を使って 失敗の原因に関する情報を取得 できます。

In [ ]:
job

In [ ]:
job.status()

In [ ]:
error_message = job.error_message()
print(error_message)

### 結果の取得と解析

ジョブが送信され、実行が完了したら、`job.result()`を使って結果を取得できます。その後、あらかじめ定義しておいたヘルパー関数を用いて結果を評価し、期待どおりの結果になっているかを確認します。

In [ ]:
# Get the job status, error affecting the job (if any), and result
result = job.result()
status = job.status()
error_message = job.error_message()
end = time.time()

print("Job status: ", status)
print("Error message: ", error_message)
print("Action status: ", result["status"])
print("Action message: ", result["message"])
print("Action result: ", result["data"])

In [ ]:
predictions = result["data"]["predictions"]
evaluate_predictions(predictions, y_test, start, end)

# Part 3: 実機量子ハードウェア上の量子分類器

### Jobの実行

次に、実際の量子ハードウェアを使って評価を行いましょう。このアプローチには多くの利点があり、その中でも学習者数が増えるにつれてスケーラビリティが向上します。シミュレーターモードが古典的なリソースを使うのに対し、実際の量子最適化はより大規模なアンサンブルやより複雑な探索空間を効率的に扱うことができます。

量子最適化を有効にするには、`optimizer_options` の`simulator`フラグを`False`に設定します。また、収束挙動を制御するための`classical_optimizer_options`パラメータも含めてください。QAOAのような量子アルゴリズムは一般的に60以上の`maxiter`値が恩恵を受けますが、デモンストレーションのためには `10` に制限し、実行時間を短縮し、約10分が期待されます。

このセクションでは、Qiskit Serverlessを用いてハイブリッド量子・古典ワークフローを大規模に実行し、研究と本番環境の量子アプリケーションの間のギャップを埋める方法を示します。


<div class="alert alert-block alert-success">
  <strong>Exercise 3:</strong>  
  <code>#TODO</code>セクションに入力して、実際の量子ハードウェア上で<code>QuantumEnhancedEnsembleClassifier</code>を実行するジョブの最適化オプションを指定します。最適化の反復回数を<code>10</code> に制限し、合理的な時間(期待:~10分)で完了させます。
</div>

<div class="alert alert-block alert-info">
  <strong>💡 Tip:</strong> <em>Part 1: Simulated Quantum Classifier</em>で使われている構成と似たものを使用してください。Exerciseにpassしたら、<code>regularization</code>、<code>num_learners</code>、<code>num_solutions</code>など、さまざまなパラメータを試してみて、それらがパフォーマンスや最適化時間にどう影響するかを探ってみてください。例として<a href="https://docs.quantum.ibm.com/guides/multiverse-computing-singularity" target="_blank">こちら</a>を参照してください。
</div>

In [ ]:
optimizer_options_ex3 = {
    ### TODO: Write your code below here ###
    
    ### Don't change any code past this line ###
}

In [ ]:
grade_exercise_3(optimizer_options_ex3)

<div class="alert alert-block alert-warning">

**⚠️ 警告：QPU時間の消費について**

以下のセルを実行すると、QPU にジョブが送信され、実際の QPU 実行時間が消費されます。実行してよいことを確認したうえで、続行してください。

**推定 QPU 実行時間:** 2分47秒（`ibm_brisbane` でのテスト結果に基づく）

</div>

In [ ]:
# Create, fit, and predict
print("-- Creating, fitting, and predicting --")
start = time.time()
job = singularity.run(
    action="create_fit_predict",
    name="classifier_for_grid_stability",
    quantum_classifier="QuantumEnhancedEnsembleClassifier",
    num_learners=75,
    regularization=15,
    optimizer_options=optimizer_options_ex3,
    backend_name=None,
    instance=your_crn,
    random_state=RANDOM_STATE,
    X_train="grid_stability_X_train.npy",
    y_train="grid_stability_y_train.npy",
    X_test="grid_stability_X_test.npy",
    fit_params={
        "validation_data": (X_val, y_val),
    },
    options={
        "save": False,
    }
)

### ジョブ実行のモニター

ジョブが送信されたので、`job` を表示して JobID を保存 できます。また、`job.status()` を使うことで 実行状況（ステータス）を確認 できます。このセルは再実行することで、最新のステータスを再取得（ポーリング） できます。もしジョブが失敗した場合は、`job.error_message()` を使って 失敗の原因に関する情報を取得 できます。

In [ ]:
job

In [ ]:
job.status()

In [ ]:
error_message = job.error_message()
print(error_message)

### 結果の取得と解析

ジョブが送信され、実行が完了したら、`job.result()`を使って結果を取得できます。その後、あらかじめ定義しておいたヘルパー関数を用いて結果を評価し、期待どおりの結果になっているかを確認します。

In [ ]:
# Get the job status, logs, and result
result = job.result()
status = job.status()
error_message = job.error_message()
end = time.time()

print("Job status: ", status)
print("Error message: ", error_message)
print("Action status: ", result["status"])
print("Action message: ", result["message"])
print("Action result: ", result["data"])

In [ ]:
predictions = result["data"]["predictions"]
evaluate_predictions(predictions, y_test, start, end)

### 可視化：古典モデルと量子モデルの F1 スコア比較

実験結果をまとめるために、古典分類器と量子分類器の F1 スコアを比較してみましょう。F1 スコアは、適合率（precision）と再現率（recall）の両方を考慮したバランスの取れた評価指標であり、このような不均衡な分類タスクにおいて特に有用です。
この可視化により、量子最適化によって得られた性能向上を明確に確認できます。

In [ ]:
# Collect F1 scores from both experiments
labels = ["AdaBoost (Classical)", "Quantum Classifier"]
f1_scores = []

# Recalculate F1 for AdaBoost
_, _, _, f1_ada = evaluate_predictions(classifier.predict(X_test), y_test)
f1_scores.append(f1_ada)

# Use F1 score from the quantum job
_, _, _, f1_quantum = evaluate_predictions(predictions, y_test)
f1_scores.append(f1_quantum)

# Create the bar chart
plt.figure(figsize=(8, 5))
plt.bar(labels, f1_scores, color=["#808080", "#8A2BE2"])
plt.ylabel("F1 Score")
plt.title("F1 Score Comparison: Classical vs Quantum Classifier")
plt.ylim(0, 1.0)
for i, f1 in enumerate(f1_scores):
    plt.text(i, f1 + 0.01, f"{f1:.3f}", ha='center', va='bottom', fontweight='bold')
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# 共有ディレクトリのクリーンアップ

すべてのリモート実行が完了したら、Qiskit Serverless 環境にアップロードした一時ファイルをクリーンアップすることが重要です。これにより、作業スペースを整理された状態に保ち、共有環境やストレージ容量が限られた環境での不要な容量消費を防ぐことができます。

このステップでは、`client.file_delete()` メソッドを使用して、トレーニングデータおよびテストデータに対応するアップロード済みの `.tar` アーカイブを削除します。これらのファイルは、ジョブの完了と結果の取得後には不要になります。

クリーンアップは、特に複数ユーザーで作業する場合や、本番環境でワークフローを自動化する場合において、良い運用習慣（ベストプラクティス）です。

In [ ]:
# Clean up the shared data directory
file_names = [
    "grid_stability_X_train.npy",
    "grid_stability_y_train.npy",
    "grid_stability_X_test.npy",
]

for name in file_names:
    tar_name = f"{name}.tar"
    client.file_delete(tar_name, singularity)
    if os.path.exists(name):
        os.remove(name)
    if os.path.exists(tar_name):
        os.remove(tar_name)

# スケーリングによる利点と拡張性

`QuantumEnhancedEnsembleClassifier` の真価は、**大規模なアンサンブルサイズ** や **高次元の最適化問題** を扱う場面で発揮されます。このラボでは、ジョブを 30 分以内に完了させるために、学習器の数や反復回数(`maxiter = 10`)を制限しました。しかし、QAOA のような量子最適化アルゴリズムは、通常、効果的に収束させるために 60 回以上の反復を必要とします。したがって、この設定は、潜在的な性能向上のほんの一端を示しているにすぎません。

古典的なアンサンブル最適化では、学習器の数が増えるにつれて学習時間のスケーリングが悪化し、多くの場合指数関数的に増大します。これは、最適な学習器の組み合わせ選択が組合せ爆発的な性質を持つためであり、大規模なアンサンブル構成では現実的ではなくなります。一方で、量子最適化は、学習器の数や問題サイズ（例：量子ビット数）を増やしても、より良好なスケーリング特性を示すことが期待されています。実際には、古典手法が計算的に実行不可能になるような場合でも、量子手法であれば効率的に処理できる可能性があります。

# フィードバックアンケート
Qiskit Function をご利用いただいた体験について、ぜひご意見をお聞かせください。
皆さまからのフィードバックは、Qiskit Function の提供者がツールやサービスを改善・強化していくうえで大変貴重です。
ぜひお時間のあるときに、約2分で完了する簡単な[フィードバックアンケート](https://airtable.com/app6VujlNUHZuOnAF/pagpw6TgP9UEt4TAT/form) にご協力ください。

# References

1. Qiskit Function:
   - [Qiskit Function Tutorial](https://docs.quantum.ibm.com/guides/functions)
   - [Qiskit Serverless Documentation](https://qiskit.github.io/qiskit-serverless/index.html)

2. Papers:
   - [Neven et al. 2008: Training a Binary Classifier with the Quantum Adiabatic Algorithm](https://arxiv.org/abs/0811.0416)
   - [Neven et al. 2012: QBoost: Large Scale Classifier Training with Adiabatic Quantum Optimization](http://proceedings.mlr.press/v25/neven12/neven12.pdf)


# Additional information

**Created by:** Sepehr Hosseini, Pablo Lauret

**Advised by:** Junye Huang

**Version:** 1.0.0

# Qiskit packages versions

In [ ]:
import qiskit
import qiskit_serverless
import qiskit_ibm_catalog

print(f'Qiskit: {qiskit.__version__}')
print(f'Qiskit Serverless: {qiskit_serverless.__version__}')
print(f'Qiskit IBM Catalog: {qiskit_ibm_catalog.__version__}')